In [4]:
from tqdm import tqdm
import argparse
from utils import prepare_answer, read_jsonl, normalize_str, get_list_str, save_2_jsonl, prepare_dataset


In [16]:
def match(answer, result):
    answer = [normalize_str(i) for i in answer]
    for ans in answer:
        if ans in normalize_str(result):  # exact match
            return True
        # else:
        #     for res in [normalize_str(i) for i in get_list_str(result)]:   # exact match with different ordering e.g. world series 2014 == 2014 world series
        #         if all([i in res for i in ans.split(' ')]):
        #             return True
    return False
   
def reverse_match(answer, result):   # e.g question: when ...? answer: 2014 world series, result: 2014
    answer = [normalize_str(i) for i in answer]
    result = [normalize_str(i) for i in get_list_str(result)]
    for res in result:
        if any([res in i for i in answer]):
            return True
    return False

In [5]:
datas, question_string = prepare_dataset('webqsp')
answers = prepare_answer('webqsp', False)
results = read_jsonl('output/lmp_webqsp_llama-2_1hop.jsonl')
# new_results = read_jsonl('lmp_cwq_llama-2_3hop_e.jsonl')
# new_index = [168, 200, 207, 229, 243, 265, 271, 321, 325, 353, 388, 399, 418, 428, 476, 488, 525, 537, 552, 569, 577, 587, 589, 597, 603, 606, 615, 657, 670, 671, 673, 678, 690, 700, 705, 709, 776, 811, 830, 865, 870, 905, 906, 1012, 1021, 1027, 1054, 1062, 1094, 1107, 1108, 1123, 1127, 1138, 1142, 1168, 1181, 1182, 1203, 1209, 1236, 1248, 1265, 1268, 1290, 1323, 1328, 1334, 1336, 1352, 1357, 1369, 1371, 1393, 1403, 1442, 1446, 1447, 1448, 1455, 1468, 1470, 1472, 1496, 1497, 1516, 1520, 1529, 1555, 1565, 1573, 1602, 1622, 1634, 1694, 1724, 1738, 1750, 1797, 1814, 1816, 1828, 1841, 1866, 1879, 1895, 1914, 1934, 1980, 1984, 1988, 2011, 2015, 2120, 2130, 2149, 2163, 2169, 2180, 2197, 2210, 2220, 2221, 2243, 2249, 2273, 2314, 2322, 2377, 2378, 2387, 2402, 2418, 2429, 2446, 2457, 2491, 2501, 2503, 2531, 2539, 2541, 2549, 2552, 2573, 2591, 2592, 2603, 2635, 2652, 2693, 2723, 2748, 2753, 2759, 2768, 2771, 2830, 2887, 2897, 2898, 2912, 2918, 2927, 2940, 2947, 2952, 2961, 2996, 3014, 3016, 3028, 3068, 3071, 3081, 3111, 3117, 3125, 3127, 3129, 3131, 3134, 3142, 3143, 3147, 3149, 3156, 3157, 3158, 3165, 3169, 3171, 3179, 3180, 3184, 3185, 3191, 3193, 3198, 3199, 3205, 3207, 3209, 3219, 3224, 3228, 3230, 3231, 3234, 3239, 3243, 3246, 3248, 3252, 3254, 3256, 3257, 3261, 3264, 3272, 3273, 3276, 3282, 3285, 3291, 3303, 3317, 3337, 3350, 3356, 3362, 3371, 3377, 3408, 3410, 3457, 3459, 3466, 3470, 3482, 3508, 3528]
# assert len(new_results)  == len(new_index)


# e_r = [69, 89, 144, 200, 207, 208, 209, 210, 222, 229, 232, 236, 238, 243, 259, 263, 265, 280, 283, 292, 294, 295, 305, 318, 321, 325, 334, 336, 347, 353, 366, 378, 388, 390, 399, 411, 418, 430, 456, 462, 470, 488, 496, 502, 505, 506, 513, 514, 520, 525, 530, 535, 536, 537, 542, 543, 545, 552, 553, 554, 555, 566, 576, 577, 578, 583, 587, 597, 603, 613, 624, 641, 666, 671, 683, 694, 705, 713, 714, 720, 726, 730, 741, 762, 788, 790, 802, 804, 811, 865, 867, 870, 905, 937, 948, 966, 974, 993, 999, 1012, 1021, 1045, 1047, 1050, 1053, 1064, 1094, 1101, 1107, 1108, 1118, 1127, 1138, 1153, 1159, 1168, 1178, 1208, 1230, 1264, 1265, 1280, 1282, 1323, 1328, 1336, 1352, 1357, 1369, 1371, 1391, 1393, 1397, 1402, 1403, 1413, 1442, 1458, 1462, 1469, 1470, 1482, 1508, 1515, 1520, 1523, 1527, 1573, 1581, 1592, 1602, 1608, 1653, 1664, 1666, 1671, 1687, 1691, 1694, 1700, 1713, 1724, 1739, 1741, 1793, 1797, 1799, 1812, 1815, 1854, 1866, 1883, 1899, 1914, 1920, 1975, 1980, 2006, 2011, 2015, 2021, 2032, 2035, 2049, 2054, 2059, 2062, 2064, 2107, 2118, 2120, 2132, 2140, 2149, 2169, 2172, 2173, 2197, 2215, 2222, 2249, 2261, 2273, 2294, 2300, 2323, 2335, 2363, 2373, 2377, 2378, 2402, 2432, 2433, 2452, 2472, 2491, 2501, 2528, 2531, 2549, 2564, 2566, 2572, 2573, 2603, 2627, 2652, 2653, 2654, 2693, 2698, 2705, 2706, 2732, 2750, 2753, 2759, 2773, 2778, 2784, 2785, 2788, 2791, 2816, 2830, 2849, 2853, 2861, 2862, 2879, 2883, 2887, 2897, 2898, 2912, 2913, 2918, 2920, 2938, 2952, 2966, 2992, 2996, 3005, 3053, 3068, 3071, 3083, 3091, 3101, 3124, 3128, 3138, 3151, 3161, 3166, 3180, 3181, 3187, 3189, 3191, 3201, 3216, 3221, 3225, 3230, 3238, 3239, 3243, 3267, 3270, 3276, 3285, 3291, 3303, 3317, 3337, 3344, 3356, 3368, 3377, 3408, 3421, 3439, 3451, 3452, 3459, 3468, 3470, 3483, 3506, 3509, 3528]
# assert len(new_results)  == len(e_r)

100%|██████████| 1639/1639 [00:00<00:00, 274265.48it/s]


In [11]:
results[0]

{'question': 'who influenced samuel taylor coleridge?',
 'result': '  Sure, here are the possible answers to your question:\n\n1. Emanuel Swedenborg\n2. Baruch Spinoza\n3. Giambattista Vico\n4. Thomas Browne\n5. Immanuel Kant\n6. Algernon Charles Swinburne\n7. Friedrich Wilhelm Joseph Schelling\n8. Robert Burns\n9. William Shakespeare\n10. John Milton',
 'path': [{'entity:': 'Samuel Taylor Coleridge',
   'relations': ['book.author.works_written',
    'influence.influence_node.influenced_by',
    'people.person.education',
    'people.person.places_lived',
    'people.person.profession']}],
 'prompt': ['Coleridge was influenced by various authors and thinkers, including Emanuel Swedenborg, Baruch Spinoza, Giambattista Vico, Thomas Browne, Immanuel Kant, Algernon Charles Swinburne, Friedrich Wilhelm Joseph Schelling, Robert Burns, William Shakespeare, and John Milton.',
  "Coleridge attended Jesus College, Cambridge, and Christ's Hospital.",
  'Coleridge lived in Highgate and Ottery St M

In [3]:
new[0]

NameError: name 'new' is not defined

In [7]:
results[0]['path']

[{'entity:': 'Samuel Taylor Coleridge',
  'relations': ['book.author.works_written',
   'influence.influence_node.influenced_by',
   'people.person.education',
   'people.person.places_lived',
   'people.person.profession']}]

In [13]:
new = [0] * len(datas)
for i, v in enumerate(datas):
    for j in results:
        if j['question'] == v[question_string]:
            path = {}
            for k in j['path']:
                path.update({k['entity:']: {}})
                for ii, vv in enumerate(k['relations']):
                    path[k['entity:']].update({vv: j['prompt'][ii]})
            new[i] = {"question": j['question'], "result": j['result'], "paths": path}

In [14]:
new

[{'question': 'who influenced samuel taylor coleridge?',
  'result': '  Sure, here are the possible answers to your question:\n\n1. Emanuel Swedenborg\n2. Baruch Spinoza\n3. Giambattista Vico\n4. Thomas Browne\n5. Immanuel Kant\n6. Algernon Charles Swinburne\n7. Friedrich Wilhelm Joseph Schelling\n8. Robert Burns\n9. William Shakespeare\n10. John Milton',
  'paths': {'Samuel Taylor Coleridge': {'book.author.works_written': 'Coleridge was influenced by various authors and thinkers, including Emanuel Swedenborg, Baruch Spinoza, Giambattista Vico, Thomas Browne, Immanuel Kant, Algernon Charles Swinburne, Friedrich Wilhelm Joseph Schelling, Robert Burns, William Shakespeare, and John Milton.',
    'influence.influence_node.influenced_by': "Coleridge attended Jesus College, Cambridge, and Christ's Hospital.",
    'people.person.education': 'Coleridge lived in Highgate and Ottery St Mary.',
    'people.person.places_lived': 'Coleridge worked as a critic, poet, and philosopher.',
    'people.

In [ ]:
new_index = []
for i in results:
    for j, v in enumerate(datas):
        if i['question'] == v[question_string]:
            new_index.append(j)

In [14]:
hits = []
for result in new:
    answer = answers[result['question']]
    if match(answer, result['result']) or reverse_match(answer, result['result']):
        hits.append(1)
        # else:
        #     hits.append(0)
    else:
        hits.append(0)

print("# of Correct: {}".format(sum(hits)))
print("# of Wrong: {}".format(len(hits)- sum(hits)))
print("Hit@1: {}".format(sum(hits) / len(hits)))
# print([i for i, v in enumerate(hits) if v == 1])

# of Correct: 1383
# of Wrong: 256
Hit@1: 0.8438071995118975


In [ ]:
for i, v in enumerate(new_index):
    if results[v]['question'] == new_results[i]['question']:
        answer = answers[results[v]['question']]
        if match(answer, new_results[i]['result']) or reverse_match(answer, new_results[i]['result']):
            results[v] = new_results[i]
        #     print(answer)
        #     print(results[v]['result'])
        # else:
            results[v]['paths'] = new_results[i]['paths']
        #     print(0)
    # else:
    #     print(i)
    #     print(v)

In [17]:
hits = []
for result in new:
    answer = answers[result['question']]
    result = result['result']
    if match(answer, result) or reverse_match(answer, result):
        hits.append(1)
    else:
        hits.append(0)
        # print(answer)
        # print(result)

print("# of Correct: {}".format(sum(hits)))
print("# of Wrong: {}".format(len(hits)- sum(hits)))
print("Hit@1: {}".format(sum(hits) / len(hits)))

# of Correct: 1383
# of Wrong: 256
Hit@1: 0.8438071995118975


In [ ]:
for i in new:
    save_2_jsonl('lmp_webqsp_llama-2_1hop.jsonl', i)

In [ ]:
print(list(set(range(len(new_index)))- set(new_index)))

In [14]:
len(new)

3531

In [ ]:
new_index == list(range(len(new_index)))
